In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Parameters
steps = 500  # Number of steps

# Seed for reproducibility
np.random.seed(42)

# Generate random steps (+1 or -1)
random_steps = np.random.choice([-1, 1], size=steps)

# Compute the random walk (cumulative sum)
position = np.cumsum(random_steps)

# Correct for drift by keeping the sum close to zero
position = position - np.mean(position)

# Initialize the plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0, steps)
ax.set_ylim(np.min(position) - 10, np.max(position) + 10)
ax.set_title("1D Random Walk with Balanced Steps")
ax.set_xlabel("Step Number")
ax.set_ylabel("Position")

# Initialize plot elements
line, = ax.plot([], [], color="blue", label="Position", lw=2)
point, = ax.plot([], [], 'ro', label="Current Position")
step_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')
ax.axhline(0, color="black", linestyle="--", linewidth=0.8, label="Origin")
ax.legend()
ax.grid()

# Update function for animation
def update(frame):
    line.set_data(np.arange(frame + 1), position[:frame + 1])  # Update line
    point.set_data([frame], [position[frame]])  # Update current point as a sequence
    step_text.set_text(f"Step {frame}: {random_steps[frame]}")  # Show step value
    return line, point, step_text

# Create the animation
# ani = FuncAnimation(fig, update, frames=np.arange(steps), interval=50, blit=True)

# # Display animation in Jupyter Notebook
# plt.rcParams["animation.embed_limit"] = 50  # Increase limit to 50MB
# HTML(ani.to_jshtml())

# plot position
plt.plot(position)

## Pair coorelation 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
L = 50  # Lattice size
max_r = L // 2  # Maximum distance to consider

# Case 1: Ordered lattice (alternating 1s and 0s)
ordered_lattice = np.array([1 if i % 2 == 0 else 0 for i in range(L)])

# Case 2: Random lattice (randomly shuffled 1s and 0s)
random_lattice = np.array([1] * (L // 2) + [0] * (L // 2))
np.random.shuffle(random_lattice)

# Case 3: Clustered lattice (all 1s followed by all 0s)
clustered_lattice = np.array([1] * (L // 2) + [0] * (L // 2))


def pair_correlation_function(lattice, max_r):
    """Compute pair correlation function g(r) for a 1D lattice."""
    N = len(lattice)
    g = np.zeros(max_r)
    for i in range(N):
        for r in range(1, max_r + 1):
            neighbor = (i + r) % N  # Periodic boundary conditions
            if lattice[i] == lattice[neighbor]:  # Same type of particle
                g[r - 1] += 1

    g /= (N * 2)  # Normalize by number of particles and periodic neighbors
    return g


# Compute g(r) for each lattice
g_ordered = pair_correlation_function(ordered_lattice, max_r)
g_random = pair_correlation_function(random_lattice, max_r)
g_clustered = pair_correlation_function(clustered_lattice, max_r)

# Plot g(r) for all cases
plt.figure(figsize=(12, 6))

# Ordered lattice
plt.plot(range(1, max_r + 1), g_ordered, marker='o', linestyle='-', color='blue', label='Ordered')

# Random lattice
plt.plot(range(1, max_r + 1), g_random, marker='x', linestyle='--', color='green', label='Random')

# Clustered lattice
plt.plot(range(1, max_r + 1), g_clustered, marker='s', linestyle='-', color='red', label='Clustered')

# Plot details
plt.xlabel("Distance (r)")
plt.ylabel("g(r)")
plt.title("Pair Correlation Function for Different Lattice Configurations")
plt.grid(alpha=0.3)
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 50  # Number of lattice sites
steps = 10**5  # Number of random walk steps
T1, T2 = 100, 1000  # Temperatures (in Kelvin)
J_red_red = -1.0  # Interaction energy (red-red)
J_blue_blue = -1.0  # Interaction energy (blue-blue)

# Boltzmann constant
k_B = 8.617333262145e-5  # eV/K

# Helper functions
def initialize_lattice():
    """Initialize the lattice with equal numbers of red (+1) and blue (-1) particles."""
    lattice = np.array([1] * (N // 2) + [-1] * (N // 2))
    np.random.shuffle(lattice)
    return lattice

def calculate_energy(lattice, J_red_blue):
    """Calculate the total energy of the system."""
    energy = 0
    for i in range(N):
        neighbor_right = (i + 1) % N  # Periodic boundary condition
        if lattice[i] == 1 and lattice[neighbor_right] == 1:
            energy += J_red_red
        elif lattice[i] == -1 and lattice[neighbor_right] == -1:
            energy += J_blue_blue
        elif lattice[i] != lattice[neighbor_right]:
            energy += J_red_blue
    return energy

def metropolis_step(lattice, J_red_blue, T):
    """Perform a single Metropolis step."""
    # Choose a random site and its neighbor
    site = np.random.randint(N)
    neighbor = (site + 1) % N  # Periodic boundary condition

    # Swap particles
    lattice[site], lattice[neighbor] = lattice[neighbor], lattice[site]

    # Calculate energy difference
    delta_E = calculate_energy(lattice, J_red_blue) - calculate_energy(lattice, J_red_blue)

    # Metropolis acceptance criterion
    if delta_E > 0 and np.random.rand() >= np.exp(-delta_E / (k_B * T)):
        # Reject the swap
        lattice[site], lattice[neighbor] = lattice[neighbor], lattice[site]

    return lattice

def simulate_random_walk(J_red_blue, T):
    """Simulate the random walk for a given interaction energy and temperature."""
    lattice = initialize_lattice()
    for _ in range(steps):
        lattice = metropolis_step(lattice, J_red_blue, T)
    return lattice

def pair_correlation_function(lattice, max_r):
    """Compute the pair correlation function g(r)."""
    g = np.zeros(max_r)
    for i in range(N):
        for r in range(1, max_r + 1):
            neighbor = (i + r) % N  # Periodic boundary conditions
            if lattice[i] == lattice[neighbor]:
                g[r - 1] += 1
    g /= (N * 2)  # Normalize
    return g

def calculate_order_parameter(lattice):
    """Calculate the order parameter \u03b7."""
    eta = np.sum((-1)**np.arange(N) * lattice) / N
    return eta

# Main execution
if __name__ == "__main__":
    max_r = N // 2

    # Simulate clustering (J_red_blue > 0) at T1 and T2
    for J_red_blue, temp, label in [(0.5, T1, "Clustering at T1"), (0.5, T2, "Clustering at T2")]:
        final_lattice = simulate_random_walk(J_red_blue, temp)
        g_r = pair_correlation_function(final_lattice, max_r)
        plt.plot(range(1, max_r + 1), g_r, label=label)

    # Simulate ordering (J_red_blue < 0) at T1
    J_red_blue = -1.5
    final_lattice_order = simulate_random_walk(J_red_blue, T1)
    g_r_order = pair_correlation_function(final_lattice_order, max_r)
    order_parameter = calculate_order_parameter(final_lattice_order)

    plt.plot(range(1, max_r + 1), g_r_order, label="Ordering at T1")

    # Visualization
    plt.xlabel("Distance (r)")
    plt.ylabel("g(r)")
    plt.title("Pair Correlation Function for Clustering and Ordering")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

    print(f"Order parameter for ordering case: {order_parameter}")


## Que Random Walk

In [ ]:
def initialize_lattice(w_lattice):
    """Initialize the lattice"""
    if w_lattice == "ordered":
        lattice = np.array([1] * (N // 2) + [-1] * (N // 2))
    elif w_lattice == "random":
        lattice = np.array([1] * (N // 2) + [-1] * (N // 2))
        np.random.shuffle(lattice)
    elif w_lattice == "clustered":
        lattice = np.array([1] * (N // 2) + [-1] * (N // 2))
    else:
        raise ValueError("Invalid lattice type. Choose 'ordered', 'random', or 'clustered'.")

    return lattice

In [ ]:
J_red_red = -1.0  # Interaction energy (red-red)
J_blue_blue = -1.0  # Interaction energy (blue-blue)
def calculate_energy(lattice, J_red_blue):
    """Calculate the total energy of the system."""
    energy = 0
    for i in range(N):
        neighbor_right = (i + 1) % N  # Periodic boundary condition
        neighbor_left = (i - 1) % N
        if lattice[i] == 1 and lattice[neighbor_right] == 1:
            energy += J_red_red
        elif lattice[i] == -1 and lattice[neighbor_right] == -1:
            energy += J_blue_blue
        elif lattice[i] != lattice[neighbor_right]:
            energy += J_red_blue
        if lattice[i] == 1 and lattice[neighbor_left] == 1:
            energy += J_red_red
        elif lattice[i] == -1 and lattice[neighbor_left] == -1:
            energy += J_blue_blue
        elif lattice[i] != lattice[neighbor_left]:
            energy += J_red_blue
    return energy

def walk_step(lattice, J_red_blue, T):
    """Perform a single Metropolis step."""
    # Choose a random site and its neighbor
    site = np.random.randint(N)
    neighbor_right = (site + 1) % N  # Periodic boundary condition
    neighbor_left = (site - 1) % N

    which_neighbor = np.random.choice([1, -1])
    # Swap particles
    # lattice[site], lattice[neighbor_right] = lattice[neighbor_right], lattice[site]
    # lattice[site], lattice[neighbor_left] = lattice[neighbor_left], lattice[site]
    if which_neighbor == 1:
        lattice[site], lattice[neighbor_right] = lattice[neighbor_right], lattice[site]
    else:
        lattice[site], lattice[neighbor_left] = lattice[neighbor_left], lattice[site]

    # Calculate energy difference
    delta_E = calculate_energy(lattice, J_red_blue) - calculate_energy(lattice, J_red_blue)

    # Metropolis acceptance criterion
    if delta_E > 0 and np.random.rand() >= np.exp(-delta_E / (k_B * T)):
        # Reject the swap
        # lattice[site], lattice[neighbor_right] = lattice[neighbor_right], lattice[site]
        # lattice[site], lattice[neighbor_left] = lattice[neighbor_left], lattice[site]
        if which_neighbor == 1:
            lattice[site], lattice[neighbor_right] = lattice[neighbor_right], lattice[site]
        else:
            lattice[site], lattice[neighbor_left] = lattice[neighbor_left], lattice[site]

    return lattice

def pair_correlation_function(lattice, max_r):
    """Compute the pair correlation function g(r)."""
    g = np.zeros(max_r)
    for i in range(N):
        for r in range(1, max_r + 1):
            neighbor = (i + r) % N  # Periodic boundary conditions
            if lattice[i] == lattice[neighbor]:
                g[r - 1] += 1
    g /= (N * 2)  # Normalize
    return g

def calculate_order_parameter(lattice):
    """Calculate the order parameter \u03b7."""
    eta = np.sum((-1)**np.arange(N) * lattice) / N
    return eta

In [ ]:
N = 50  # Number of lattice sites
max_r = N // 2  # Maximum distance to consider
steps = 10**5  # Number of random walk steps

def simulate_random_walk(w_lattice, J_red_blue, T):
    """Simulate the random walk for a given interaction energy and temperature."""
    lattice = initialize_lattice(w_lattice)
    for _ in range(steps):
        lattice = walk_step(lattice, J_red_blue, T)
    return lattice

fin_ordered = simulate_random_walk("random", -1.5, 100)

# plot lattice with red and blue dots
plt.figure(figsize=(12, 6))
plt.scatter(range(N), fin_ordered, c=fin_ordered, cmap='coolwarm', s=100)
plt.xlabel("Site")
plt.ylabel("Particle Type")
plt.title("Final Lattice Configuration (Ordered)")
plt.grid(axis='y')
plt.show()
